# سیمینٹک کرنل کے ساتھ اوپن بی این بی ایم سی پی سرور کا انضمام

یہ نوٹ بک دکھاتی ہے کہ سیمینٹک کرنل کو اصل اوپن بی این بی ایم سی پی سرور کے ساتھ استعمال کرتے ہوئے ایم سی پی اسٹوڈیو پلگ ان کے ذریعے حقیقی ایئر بی این بی رہائش گاہوں کی تلاش کیسے کی جا سکتی ہے۔ ایل ایل ایم تک رسائی کے لیے، یہ ایزور اے آئی فاؤنڈری کا استعمال کرتی ہے۔ اپنے ماحول کے متغیرات کو ترتیب دینے کے لیے، آپ [سیٹ اپ سبق](/00-course-setup/README.md) کی پیروی کر سکتے ہیں۔


## ضروری پیکجز درآمد کریں


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## ایم سی پی پلگ اِن کنکشن بنانا

ہم [اوپن بی این بی ایم سی پی سرور](https://github.com/openbnb-org/mcp-server-airbnb) سے ایم سی پی اسٹینڈرڈ پلگ اِن کے ذریعے جڑیں گے۔ یہ سرور @openbnb/mcp-server-airbnb پیکیج کے ذریعے ایئر بی این بی تلاش کی سہولت فراہم کرتا ہے۔


## کلائنٹ بنانا

اس مثال میں، ہم اپنے LLM تک رسائی کے لیے Azure AI Foundry کا استعمال کریں گے۔ یقینی بنائیں کہ آپ کے ماحول کے متغیرات درست طریقے سے ترتیب دیے گئے ہیں۔


## ماحول کی تشکیل

Azure OpenAI کی ترتیبات کو ترتیب دیں۔ یقینی بنائیں کہ آپ کے پاس درج ذیل ماحول متغیرات موجود ہیں:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## اوپن بی این بی ایم سی پی انٹیگریشن کو سمجھنا

یہ نوٹ بک **حقیقی اوپن بی این بی ایم سی پی سرور** سے جڑتی ہے جو اصل ایئربی این بی سرچ کی سہولت فراہم کرتا ہے۔

### یہ کیسے کام کرتا ہے:

1. **MCPStdioPlugin**: ایم سی پی سرور کے ساتھ معیاری ان پٹ/آؤٹ پٹ کمیونیکیشن استعمال کرتا ہے  
2. **حقیقی این پی ایم پیکج**: `@openbnb/mcp-server-airbnb` کو npx کے ذریعے ڈاؤنلوڈ اور چلاتا ہے  
3. **لائیو ڈیٹا**: ایئربی این بی کے APIs سے اصل پراپرٹی ڈیٹا واپس کرتا ہے  
4. **فنکشن ڈسکوری**: ایجنٹ خودکار طور پر ایم سی پی سرور سے دستیاب فنکشنز دریافت کرتا ہے  

### دستیاب فنکشنز:

اوپن بی این بی ایم سی پی سرور عام طور پر یہ فنکشنز فراہم کرتا ہے:
- **search_listings** - مقام اور معیار کے مطابق ایئربی این بی پراپرٹیز تلاش کریں  
- **get_listing_details** - مخصوص پراپرٹیز کی تفصیلی معلومات حاصل کریں  
- **check_availability** - مخصوص تاریخوں کے لیے دستیابی چیک کریں  
- **get_reviews** - پراپرٹیز کے لیے ریویوز حاصل کریں  
- **get_host_info** - پراپرٹی کے میزبانوں کی معلومات حاصل کریں  

### ضروریات:

- آپ کے سسٹم پر **Node.js** انسٹال ہو  
- ایم سی پی سرور پیکج ڈاؤنلوڈ کرنے کے لیے **انٹرنیٹ کنکشن**  
- **NPX** دستیاب ہو (یہ Node.js کے ساتھ آتا ہے)  

### کنکشن کی جانچ:

آپ ایم سی پی سرور کو دستی طور پر چلا کر جانچ سکتے ہیں:
```bash
npx -y @openbnb/mcp-server-airbnb
```

یہ اوپن بی این بی ایم سی پی سرور کو ڈاؤنلوڈ اور شروع کرے گا، جس سے سیمینٹک کرنل حقیقی ایئربی این بی ڈیٹا کے لیے جڑتا ہے۔  


## اوپن بی این بی ایم سی پی سرور کے ساتھ ایجنٹ چلانا

اب ہم اے آئی ایجنٹ چلائیں گے جو اوپن بی این بی ایم سی پی سرور سے جڑتا ہے تاکہ اسٹاک ہوم میں 2 بالغوں اور 1 بچے کے لیے حقیقی ایئربی این بی رہائش تلاش کی جا سکے۔ تلاش کے معیار کو تبدیل کرنے کے لیے `user_inputs` کی فہرست کو تبدیل کرنے میں آزاد محسوس کریں۔


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

خلاصہ  
مبارک ہو! آپ نے کامیابی کے ساتھ ایک AI ایجنٹ تیار کر لیا ہے جو ماڈل کانٹیکسٹ پروٹوکول (MCP) کا استعمال کرتے ہوئے حقیقی دنیا کی رہائش کی تلاش کے ساتھ جڑتا ہے:

استعمال شدہ ٹیکنالوجیز:  
- سیمینٹک کرنل - Azure OpenAI کے ساتھ ذہین ایجنٹس بنانے کے لیے  
- Azure AI Foundry - LLM صلاحیتوں اور چیٹ مکمل کرنے کے لیے  
- MCP (ماڈل کانٹیکسٹ پروٹوکول) - معیاری ٹول انضمام کے لیے  
- OpenBnB MCP سرور - حقیقی Airbnb تلاش کی فعالیت کے لیے  
- Node.js/NPX - بیرونی MCP سرور چلانے کے لیے  

آپ نے کیا سیکھا:  
- MCP انضمام: سیمینٹک کرنل ایجنٹس کو بیرونی MCP سرورز سے جوڑنا  
- حقیقی وقت میں ڈیٹا تک رسائی: لائیو APIs کے ذریعے اصل Airbnb پراپرٹیز کی تلاش  
- پروٹوکول کمیونیکیشن: ایجنٹ اور MCP سرور کے درمیان stdio کمیونیکیشن کا استعمال  
- فنکشن دریافت: MCP سرورز سے دستیاب فنکشنز کو خودکار طور پر دریافت کرنا  
- اسٹریمنگ جوابات: حقیقی وقت میں فنکشن کالز کو کیپچر اور لاگ کرنا  
- HTML رینڈرنگ: ایجنٹ کے جوابات کو اسٹائل شدہ ٹیبلز اور انٹرایکٹو ڈسپلے کے ساتھ فارمیٹ کرنا  

اگلے اقدامات:  
- اضافی MCP سرورز کو ضم کریں (موسم، پروازیں، ریستوران)  
- MCP اور A2A پروٹوکولز کو یکجا کرتے ہوئے ایک ملٹی ایجنٹ سسٹم بنائیں  
- اپنے ڈیٹا ذرائع کے لیے کسٹم MCP سرورز بنائیں  
- سیشنز کے درمیان مستقل گفتگو کی یادداشت کو نافذ کریں  
- MCP سرور آرکیسٹریشن کے ساتھ ایجنٹ کو Azure Functions پر تعینات کریں  
- صارف کی توثیق اور بکنگ کی صلاحیتیں شامل کریں  

MCP آرکیٹیکچر کے اہم فوائد:  
- معیاری کاری: AI ایجنٹس کو بیرونی ٹولز سے جوڑنے کے لیے یونیورسل پروٹوکول  
- حقیقی وقت کا ڈیٹا: مختلف سروسز سے لائیو، تازہ ترین معلومات تک رسائی  
- توسیع پذیری: نئے ڈیٹا ذرائع اور ٹولز کو آسانی سے ضم کرنا  
- باہمی عملیت: مختلف AI فریم ورکس اور ایجنٹ پلیٹ فارمز کے ساتھ کام کرنا  
- خدشات کی علیحدگی: AI منطق اور بیرونی ڈیٹا تک رسائی کے درمیان واضح فرق  



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے پوری کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
